# Рекомендация тарифов

## Откройте и изучите файл

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from joblib import dump
from sklearn.metrics import accuracy_score

In [3]:
#
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

На "Ултра" тариф переходит чуть меньше половины пользователей

## Разбейте данные на выборки

### Разделим исходные данные на обучающую, валидационную и тестовую выборки

Разделим на целевые признаки и признаки

In [7]:
features = df.drop(['is_ultra'], axis=1) #признаки
target = df['is_ultra'] #целевое значение

In [8]:
features_trainvalid, features_test, target_trainvalid, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345)
# Сначала исходные данные разбили на две выборки - объединенную обучающую и валидационную выборки 
features_train, features_valid, target_train, target_valid = train_test_split(
    features_trainvalid, target_trainvalid, test_size=0.25, random_state=12345)

## Исследуйте модели

### Дерево решений 
<br> С помощью цикла подберем лучший гиперпараметр maxdepth

In [9]:
best_dt_model = None
best_dt_result = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_dt_result:
        best_dt_model = model
        best_dt_result = result

print("Accuracy лучшей модели:", best_dt_result)
    
print("Максимальная глубина:", depth)




Accuracy лучшей модели: 0.7744945567651633
Максимальная глубина: 10


### Случайный лес
<br>  С помощью цикла подберем лучший гиперпараметр 

In [10]:
best_forest_model = None
best_forest_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators= est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_forest_result:
        best_forest_model = model
        best_forest_result = result
print("Accuracy наилучшей модели на валидационной выборке:", best_forest_result,"Количество деревьев:", est,)

Accuracy наилучшей модели на валидационной выборке: 0.7884914463452566 Количество деревьев: 10


### Логистическая регрессия 
<br> Установим solver='lbfgs', max_iter=1000

In [11]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
model.score(features_train, target_train)
prediction_valid = model.predict(features_valid)
accuracy = accuracy_score(prediction_valid, target_valid)

print("Accuracy наилучшей модели на валидационной выборке:", accuracy)

Accuracy наилучшей модели на валидационной выборке: 0.7262830482115086


### Вывод:
Наилучшими гиперпараметрами являются: 
1. DesicionTreeClassifier при depth = 10, 
2. RandomForestClassifier при n_estimators = 50, 
3. LogisticRegression при solver = 'lbfgs', max_iter = 1000        

## Проверьте модель на тестовой выборке

Проверим лучшую модель случайного леса на тестовой выборке

In [12]:
test_forest_predictions = best_forest_model.predict(features_test)
test_forest_accuracy = accuracy_score(target_test, test_forest_predictions)
print("Accuracy лучшей модели случайного леса на тестовой выборке:", test_forest_accuracy)

Accuracy лучшей модели случайного леса на тестовой выборке: 0.7869362363919129


Проверим лучшую модель дерева решений на тестовой выборке

In [13]:
test_dt_predictions = best_dt_model.predict(features_test)
test_dt_accuracy = accuracy_score(target_test, test_dt_predictions)
print("Accuracy лучшей модели дерева решений на тестовой выборке:", test_dt_accuracy)

Accuracy лучшей модели дерева решений на тестовой выборке: 0.7884914463452566


## (бонус) Проверьте модели на адекватность

In [14]:
model = LogisticRegression()
model.fit(features_train, target_train)

# Оценка точности на валидационной выборке
accuracy = model.score(features_valid, target_valid)
print('Accuracy on validation set:', accuracy)

Accuracy on validation set: 0.7262830482115086


In [15]:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators= est) # обучите модель с заданным количеством деревьев
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid) # посчитайте качество модели на валидационной выборке
    if result > best_result:
        best_model = model
        best_result = result

print("Accuracy наилучшей модели на тестовой выборке:", best_result)

Accuracy наилучшей модели на тестовой выборке: 0.7884914463452566


Через случай лес получилось добиться accuracy не меньше 0.75 на валидационной выборке 